<a href="https://colab.research.google.com/github/eiki-code/Text_cls/blob/main/svm_matome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



---

**Step.1 文章のベクトル化**


Mecab(分かち書き)→特徴語辞書作成→BoW→TFIDF→LSI→ベクトル変換


基本的にはgensimのコーパスを使う


辞書中の単語IDと、頻度のタプルにが作られるのでそれを特徴ベクトルに変換して用いる


gensim参考：


https://kento1109.hatenablog.com/entry/2017/11/15/230909,


https://qiita.com/yasunori/items/31a23eb259482e4824e2,


https://qiita.com/3000manJPY/items/a0652d488ce3c956613d,


https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html,


https://radimrehurek.com/gensim/matutils.html,


https://www.kkaneko.jp/ai/win/bowj.html,

In [ ]:
!pip install mecab-python3 unidic-lite
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null
!pip install mecab-python3 > /dev/null

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive

drive.mount("/content/drive/")

base_path = "/content/drive/My Drive/bert_book/"

Mounted at /content/drive/


In [ ]:
import MeCab

def doc2word_list(doc):
  tagger = MeCab.Tagger('-Ochasen')
  tagger.parse('')
  node = tagger.parseToNode(doc)

  word_list = []
  while node:
    meta = node.feature.split(',')
    if meta[0] == '名詞':
      word_list.append(meta[6])
    node = node.next

  return word_list

In [ ]:
import pandas as pd
csv_path = base_path + "csv/"
train_data = pd.read_csv(csv_path+"train_data.csv", header=None, encoding="utf-8")
train_data.columns = ["text", "label"]
train_data_list = train_data["text"].values.tolist()
train_data

,text,label
0,お盆に帰省した沙織さん（30歳）に「実家は楽しかった？」と聞くと、「楽しいなんてとんでもない...,3
1,前篇が観客動員数150万人を突破し、大ヒットしている映画『僕等がいた』。4月21日には、つい...,3
2,いよいよ、差し迫ったバレンタインデー。彼にあげるだけでなく、自分でも美味しいチョコレートを味...,7
3,父親にとって、娘は目に入れても痛くないと思わせるほど、愛おしく、大切な存在だ。誕生し、成長し...,6
4,これまで数々のスーパーヒーローを生み出してきたマーベルが、わずか一本の映画のために最強のヒー...,0
...,...,...
5767,この季節の代表的な花と言えば紫陽花だ。土壌の状態（ph度）で花の色が変わり「酸性なら青」「ア...,6
5768,vol.5：東京にいながら、小旅行気分を味わえる深大寺Camera；FUJIFILM KLA...,5
5769,15日、選抜高校野球大会に21世紀枠から出場する宮城県の石巻工業高等学校が、総額約5000万...,8
5770,「ナマ足」という言葉などない頃は、大人の女がスカートの下にストッキングを履くのは当たり前だっ...,3


In [ ]:
test_data = pd.read_csv(csv_path+"test_data.csv", header=None, encoding="utf-8")
test_data.columns = ["text", "label"]
test_data_list = test_data["text"].values.tolist()
test_data

,text,label
0,推定年収5000万円と伝えられながらも、母親が生活保護を受給していると報道されていた人気お笑...,8
1,美枝さん（IT関係・37歳）は婚活ブームの初期の頃、同僚に誘われて大人数制の婚活パーティーに...,3
2,アクション・ドラマの傑作『レオン』から18年、その遺伝子を受け継ぐ新たなヒロインが誕生！『レ...,0
3,トムがイェーーイ！！全米ロック史上最強のミュージカルを映画化した『ロック・オブ・エイジズ』が...,0
4,“人を好きになる気持ち”ってどんな気持ちだっけ？最後に“恋”をしたのはいつだろう？年を重ねる...,3
...,...,...
1920,潤沢な大容量RAMと内蔵メモリを搭載した超快適動作のスマートフォン！ 既報通り、NTTドコモ...,1
1921,インターネットやパソコンは若者やビジネスパーソンだけのものではない。電通が、シニア層のネット...,4
1922,バルセロナ五輪メダリストで、現在はプロレスラーとして活躍する暴走王・小川直也が参院選を表明し...,2
1923,2008年12月に成田—ゴールドコースト線、成田—ケアンズ線に就航して以来、昨年で1周年を迎...,7


In [ ]:
# BoWを適用するためにword_listを作成
# https://qiita.com/yasunori/items/31a23eb259482e4824e2

words_list = list(map(doc2word_list, train_data_list))
print(words_list[0])

['お盆', '帰省', '沙織', 'さん', '*', '歳', '実家', 'とんでも', '顔', '実家', '親戚', 'たち', '口々', '結婚', '母', '祖母', '結婚', '言葉', 'みんな', '私', '結婚', '楽しみ', 'の', 'ん', 'それ', 'プレッシャー', 'ん', '結婚', 'もの', '結婚', '沙織', 'さん', 'よう', '婚', '活', '中', '独', '女', 'プレッシャー', 'よう', '独', '女', '母', '正美', 'さん', '*', '歳', '世間', '娘', 'さん', '結婚', '仕事', '婉曲', '仕事', '楽', '職場', '無遠慮', '言葉', '余計', 'お世話', 'お節介', '何', '度', 'そう', '人', '心', '領域', '土足', 'よう', '真似', 'ん', '正美', 'さん', '首', '人', '嫌', 'こと', '人', 'こと', '純粋', '好奇', '心', '無神経', 'こと', '人', 'うち', '課', '定年', '男性', '社員', 'ん', '他', '課', '社員', '昼休み', '声', '定年', '後', 'の', '就職', '先', 'みたい', '気の毒', '気', '洋子', 'さん', '*', '歳', '父', '入院', '会社', '先輩', '病名', '癌', '入院', '後悔', '声', '本人', '悪意', '方', '困惑', '場合', '今年', '会社', '員', 'まゆ', 'かさん', '*', '歳', '法事', '席', '久しぶり', '父方', '伯母', '給料', 'そう', '側', '父', '人', 'よう', '額', '代わり', 'それ', '以上', 'こと', '伯母', '母', '父', 'ボーナス', '額', 'ん', 'いくら', '親戚', 'ひと', '財布', '*', 'まゆ', 'かさん', '憤慨', '北京', '五輪', '前', '市民', 'マナー', '向上', '北京', '当局', '外国', '人', '*', '質問', 'の', '個

ここまでで分かち書きされた文章のリスト（2次元）がtest_csvのデータに対して作成完了



---



---



In [ ]:
from gensim import corpora

# 特徴語辞書を作る
dic = corpora.Dictionary(words_list)
dic.filter_extremes(no_below = 20, no_above = 0.3)
bow_corpus = [dic.doc2bow(d) for d in words_list]
# 辞書の保存(未知文書分類のため)
dic.save_as_text('/content/drive/My Drive/svm_cls/testdic.txt')


ここまでがBoW


---


---



In [ ]:
from gensim import models

tfidf_model = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf_model[bow_corpus]
#tf-idfモデルの保存(未知文書分類のため)
tfidf_model.save('/content/drive/My Drive/svm_cls/tfidf_model.model')

In [ ]:
from gensim import models

# LSI次元圧縮
lsi_model = models.LsiModel(tfidf_corpus, id2word = dic, num_topics = 300)
lsi_corpus = lsi_model[tfidf_corpus]
#lsiモデルの保存(未知文書分類のため)
lsi_model.save('/content/drive/My Drive/svm_cls/lsi_model.model')

# corpusはgensimで入力として使うための形式

In [ ]:
i= 0
for doc, as_text in zip(lsi_corpus, train_data["text"]):
    print(doc, as_text)
    i+=1

    if i >= 5:
      break

[(0, 0.20646865950835094), (1, 0.16407384904754446)] お盆に帰省した沙織さん（30歳）に「実家は楽しかった？」と聞くと、「楽しいなんてとんでもない」と険しい顔が返ってきた。実家に集まった親戚たちからは口々に「結婚はまだか」と聞かれ、母や祖母には結婚を急かす言葉まで浴びせられた。「みんなが私の結婚を楽しみにしているのは分かるんですけど、それってすごいプレッシャーなんです。焦っても結婚できるものではないじゃないですか」「結婚はまだ？」という問いかけは、沙織さんのような婚活中の独女にはものすごいプレッシャーになるようだが、独女の母、正美さん（56歳）も世間からの「娘さん、結婚はまだ？」には悩まされている。「仕事が忙しいらしくて」と婉曲に答えれば、「仕事を辞めさせたら」とか「楽な職場に変われば」と無遠慮な言葉まで投げかけられ、「余計なお世話、大きなお節介です」と何度も叫びそうになったとか。「人って、どうして踏みこんでほしくない心の領域に、土足で入ってくるような真似をするんでしょうか」と正美さんは首を傾げる。「人に聞かれて嫌なことは人にも聞かない」とはよく言われることだが、純粋な好奇心から無神経なことを聞く人はいる。「うちの課でもうすぐ定年の男性社員がいるんですが、他の課の社員が昼休みに大きな声で、定年後どうするのって聞いていました。再就職先も決まっていないみたいだったので、何だかお気の毒な気がしました」と洋子さん（32歳）。「父が入院したと会社で話したら先輩から病名を聞かれて、癌だと言いたくなくて、入院を話さなければよかったと後悔しました」という声もあった。聞いた本人に悪意はないだけに、聞かれた方は困惑する場合は多い。今年から会社員になったまゆかさん（23歳）は、法事の席で久しぶりに会った父方の伯母から「給料はいくらもらっているの？」と聞かれたそうだ。「側にいた父が、人に誇れるような額はもらっていないからと代わりに答えてくれたので、それ以上聞かれることはありませんでしたが、その伯母は母にも父のボーナスの額を聞いたらしいんです。いくら親戚でもひとのお財布を覗きこむようなことはしてはいけないんじゃないですか！」とまゆかさんは憤慨していた。北京五輪を前に市民のマナー向上に北京当局がまとめた「外国人に聞いてはいけない8つの質問」と

In [ ]:
from gensim import corpora,matutils

print(lsi_corpus[0])
dense = list(matutils.corpus2dense([lsi_corpus[0]], num_terms=len(dic)).T[0])
print(len(dense))

[(0, 0.20646865950835094), (1, 0.16407384904754446)]
5198


tfidfとLSIがここまで（この時点ではまだタプル形式）



---



---


ここでmatutils.corpus2dense()を用いて特徴ベクトルに変換


使い方：denseがベクトルになる


from gensim import corpora,matutils


dense = list(matutils.corpus2dense([lsi_corpus[0]],
num_terms=len(dic)).T[0])

In [ ]:
from gensim import corpora,matutils
def vec2dense(vec, num_terms):
    return list(matutils.corpus2dense([vec], num_terms=num_terms).T[0])
len_lsi = len(list(lsi_corpus))
train_doc_vec  = [vec2dense(lsi_corpus[i],len(dic)) for i in range(len_lsi)]
print(len(train_doc_vec))

5772


これでインプットに使える形式になった


---



---


**Step.2 分類器を訓練してテストしてみる**

In [ ]:
# 訓練データの正解ラベルを用意
train_label = list(train_data["label"])
print(len(train_label))

5772


In [ ]:
from sklearn import svm
import joblib # sklearn.externalsにバインドしなくて大丈夫

# コストパラメータやRBFカーネルパラメータはgrid_searchによって最適なパラメータを選択
clf = svm.SVC(C = 10, gamma = 0.1, kernel = 'rbf')
# train_docはベクトル化した文書のリスト
# labelsは文書のカテゴリのリスト
clf.fit(train_doc_vec, train_label)
#分類器の保存
joblib.dump(clf, '/content/drive/My Drive/svm_cls/svm.pkl.cmp', compress = True)

['/content/drive/My Drive/svm_cls/svm.pkl.cmp']

In [ ]:
#　テストデータのvec化 test_data_listの１つ目のデータに対して試してみる

test_words_list = list(map(doc2word_list, test_data_list))

print(test_words_list[0])

['推定', '年収', '*', '万', '円', '母親', '生活', '保護', '受給', '報道', '人気', 'お笑い', 'タレント', '*', '日', '吉本興業', '河本', '準一', '一部', '報道', '名前', '公表', '上', '現在', '生活', '保護', '受給', '噂', '否定', 'ネット', '掲示板', '波紋', '吉本興業', '側', '発表', 'リリース', '重大', 'プライバシー', '情報', '報道', 'こと', '自体', '重大', '人権', '侵害', '生活', '保護', '費', '支給', '河本', '無名', '時代', '開始', 'もの', '現在', '生活', '保護', '費', '受給', '片山', '氏', '国会', '議員', 'ら', '問題', '言及', 'こと', '河本', '親族', '側', '事情', '十分', '確認', 'まま', '実名', '個人', '批判', '的', '発言', 'こと', '非常', 'こと', 'ネット', '掲示板', '生活', '保護', 'の', '事実', 'ん', '問題', 'いつ', 'いつ', 'いつ', '生活', '保護', 'の', 'それ', '疑惑', '河本', '母親', '仕送り', '時期', '生活', '保護', '時期', '一', '発', 'アウト', '疑惑', '否定', '具体', '性', '吉本興業', '発表', '納得', 'ユーザ', 'コメント', 'その他', '世間', 'くせ', '謝罪', '言葉', '公表', 'の', '一部', '吉本興業', '対応', '自体', '不満', '声', '関連', '記事', '吉本', '完全', '否定', '河本', '母', '生活', '保護', '不正', '受給', '疑惑', '河本', '準一', '生活', '保護', '受給', '母', '言い分', '正当', '性', '次長', '課長', '河本', '準一', '逮捕', '危機', '*', '生活', '保護', '不正', '受給', '疑惑', '片山', 'さつき', '議員', 

In [ ]:
test_bow_corpus = [dic.doc2bow(test_words_list[1])] # テストデータの中で2行目のものを試してみる

test_tfidf_corpus = tfidf_model[test_bow_corpus]

test_lsi_corpus = lsi_model[test_tfidf_corpus]

test_len_lsi = len(list(test_lsi_corpus))
test_doc_vec  = [vec2dense(test_lsi_corpus[i],len(dic)) for i in range(test_len_lsi)]
print(len(test_doc_vec))

1


In [ ]:
from sklearn import svm
import joblib

clf = joblib.load('/content/drive/My Drive/svm_cls/svm.pkl.cmp')
predicted = clf.predict(test_doc_vec)

# label_cat_dictはラベルとカテゴリのペア
print("分類結果:", str(predicted[0]))

分類結果: 3


In [ ]:
print(test_data.iat[1,0])
print(test_data.iat[1,1])

美枝さん（IT関係・37歳）は婚活ブームの初期の頃、同僚に誘われて大人数制の婚活パーティーに何度か参加した。しかし、自己紹介の連続に加え「無口な男性が多くて、こちらが会話の糸口を探してばかりで……」すっかり疲れたという。同じような会話の繰り返しだったため、フリータイムで声をかけてくれた人とも最初の会話の記憶が薄く話がはずまなかった。結局、同じようなパターンが続き婚活には見切りをつけてしまった。そんな美枝さんに友人は、「それなら、あなたが好きな歴史やゲームを通じた婚活はどう？ パーティーじゃなくても、まずネットで交流するとか」と薦めてくれた。しかし美枝さんは、「うんちく好きな人が多そうだし、私はそんなに専門的なわけでもないし」と気乗りしなかった。またもや会話のキャッチボールが心配なようだ。だが、美枝さんの友人のアドバイスも一考すべきだろう。趣味の集まりなら共通の話題で話もはずみそうだ。また婚活パーティーでは年齢設定が決まっていることが多く、年齢が参加のネックになることもあるが、趣味の会なら最初はあまり年齢も意識しなくて済む。そもそも、ずいぶん前からネットで趣味を通じて知り合い、結婚したという「ネット婚」はあった。体を動かすことが好きな有希さん（事務・当時37歳）の場合は、ニフティのコミュニティでインラインスケートのサークルを知り「やったことはないけれど面白そう！」と、公園での練習会に参加し始めた。ほどなく、サークル仲間と結婚。誠実そうな彼にアタックする人は何人もいたが、有希さんの粘り勝ちだったそうだ。「あまり自分から積極的になるタイプではなかったのですが、ライバルが多かったのが刺激になったのかも」と有希さん。得意分野を活かしてフリーでweb制作や翻訳などをしながらヴォーカリストを目指していた真実さん（当時29歳）も、インターネットで知り合った音楽活動のパートナーと、実生活でもパートナーに。彼が作曲、彼女が作詞とヴォーカルを担当。創作面の相性の良さから恋愛に発展するのに時間はかからなかったそうだ。残念ながら音楽活動は収入のメインにはなっていないが、インディーズ活動も続けている。ネットでの出会いは危険？ という見方が多かった頃でも「安心して会うことができた」のは、「会う前からだいたいどんな人か分かっていましたから。メールなどの文章って、人柄がすごく表れるじゃないですか」と

**うまくいった**